In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from pysr import PySRRegressor

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


In [2]:
import scipy.io
from scipy.interpolate import griddata

In [3]:
import pandas as pd

In [4]:
data = scipy.io.loadmat(r'data\burgers.mat')

In [5]:
data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: MACI64, Created on: Sat Aug 27 07:36:14 2016',
 '__version__': '1.0',
 '__globals__': [],
 't': array([[ 0. ],
        [ 0.1],
        [ 0.2],
        [ 0.3],
        [ 0.4],
        [ 0.5],
        [ 0.6],
        [ 0.7],
        [ 0.8],
        [ 0.9],
        [ 1. ],
        [ 1.1],
        [ 1.2],
        [ 1.3],
        [ 1.4],
        [ 1.5],
        [ 1.6],
        [ 1.7],
        [ 1.8],
        [ 1.9],
        [ 2. ],
        [ 2.1],
        [ 2.2],
        [ 2.3],
        [ 2.4],
        [ 2.5],
        [ 2.6],
        [ 2.7],
        [ 2.8],
        [ 2.9],
        [ 3. ],
        [ 3.1],
        [ 3.2],
        [ 3.3],
        [ 3.4],
        [ 3.5],
        [ 3.6],
        [ 3.7],
        [ 3.8],
        [ 3.9],
        [ 4. ],
        [ 4.1],
        [ 4.2],
        [ 4.3],
        [ 4.4],
        [ 4.5],
        [ 4.6],
        [ 4.7],
        [ 4.8],
        [ 4.9],
        [ 5. ],
        [ 5.1],
        [ 5.2],
        [

In [6]:
# Extract variables
t = data['t'].flatten()  # 1D array for time
x = data['x'].flatten()  # 1D array for space
usol = np.real(data['usol'])  # Take only the real part if `usol` is complex

In [7]:
# Create a meshgrid for all space-time combinations
T, X = np.meshgrid(t, x, indexing='ij')

In [8]:
t

array([ 0. ,  0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ,
        1.1,  1.2,  1.3,  1.4,  1.5,  1.6,  1.7,  1.8,  1.9,  2. ,  2.1,
        2.2,  2.3,  2.4,  2.5,  2.6,  2.7,  2.8,  2.9,  3. ,  3.1,  3.2,
        3.3,  3.4,  3.5,  3.6,  3.7,  3.8,  3.9,  4. ,  4.1,  4.2,  4.3,
        4.4,  4.5,  4.6,  4.7,  4.8,  4.9,  5. ,  5.1,  5.2,  5.3,  5.4,
        5.5,  5.6,  5.7,  5.8,  5.9,  6. ,  6.1,  6.2,  6.3,  6.4,  6.5,
        6.6,  6.7,  6.8,  6.9,  7. ,  7.1,  7.2,  7.3,  7.4,  7.5,  7.6,
        7.7,  7.8,  7.9,  8. ,  8.1,  8.2,  8.3,  8.4,  8.5,  8.6,  8.7,
        8.8,  8.9,  9. ,  9.1,  9.2,  9.3,  9.4,  9.5,  9.6,  9.7,  9.8,
        9.9, 10. ])

In [9]:
T

array([[ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
       [ 0.1,  0.1,  0.1, ...,  0.1,  0.1,  0.1],
       [ 0.2,  0.2,  0.2, ...,  0.2,  0.2,  0.2],
       ...,
       [ 9.8,  9.8,  9.8, ...,  9.8,  9.8,  9.8],
       [ 9.9,  9.9,  9.9, ...,  9.9,  9.9,  9.9],
       [10. , 10. , 10. , ..., 10. , 10. , 10. ]])

In [10]:
usol

array([[ 2.22044605e-16, -4.31008024e-11,  4.77340778e-10, ...,
         1.81398207e-04,  1.90083736e-04,  1.99013820e-04],
       [ 4.85722573e-16,  4.28848658e-11, -4.78508736e-10, ...,
         2.08914676e-04,  2.18644653e-04,  2.28635026e-04],
       [ 1.04083409e-15, -4.26225071e-11,  4.79589604e-10, ...,
         2.40188515e-04,  2.51064261e-04,  2.62217886e-04],
       ...,
       [-7.80625564e-17,  4.34146816e-11, -4.73120376e-10, ...,
         1.17517897e-04,  1.23619041e-04,  1.29915917e-04],
       [-1.82145965e-17, -4.33821173e-11,  4.74640424e-10, ...,
         1.36050551e-04,  1.42928147e-04,  1.50017788e-04],
       [ 2.08166817e-17,  4.32728853e-11, -4.76044357e-10, ...,
         1.57232607e-04,  1.64970064e-04,  1.72934908e-04]])

In [11]:
# Flatten the arrays to create a table
df = pd.DataFrame({
    't': T.flatten(),       # Time values
    'x': X.flatten(),       # Spatial values
    'u': usol.flatten()     # Solution values
})

In [14]:
# Save the dataframe for PyRegressor
df.to_csv("data/burgers_data.csv", index=False)

In [12]:
df

,t,x,u
0,0.0,-8.0000,2.220446e-16
1,0.0,-7.9375,-4.310080e-11
2,0.0,-7.8750,4.773408e-10
3,0.0,-7.8125,8.872463e-10
4,0.0,-7.7500,2.300882e-09
...,...,...,...
25851,10.0,7.6875,1.424345e-04
25852,10.0,7.7500,1.497234e-04
25853,10.0,7.8125,1.572326e-04
25854,10.0,7.8750,1.649701e-04


In order to discover PDEs in the data using symbolic regression, we need to compute spatial and temporal derivatives *u(x,t)*.
we can use either finite differences method or more accurately make use of Neural Networks to compute the derivatives.

**Using Finite Differences**

In [13]:
# Calculate the spatial and temporal step sizes
dx = x[1] - x[0]  # Assuming uniform spacing
dt = t[1] - t[0]  # Assuming uniform spacing

In [14]:
# First-order derivatives
u_x = np.gradient(usol, axis=1, edge_order=2) / dx  # du/dx
u_t = np.gradient(usol, axis=0, edge_order=2) / dt  # du/dt

In [15]:
# Second-order spatial derivatives
u_xx = np.gradient(u_x, axis=1, edge_order=2) / dx  # d²u/dx²

In [16]:
# Flatten the arrays and organize into a DataFrame
df_2 = pd.DataFrame({
    't': T.flatten(),
    'x': X.flatten(),
    'u': usol.flatten(),
    'u_t': u_t.flatten(),
    'u_x': u_x.flatten(),
    'u_xx': u_xx.flatten(),
})

In [21]:
# Save the DataFrame for use in symbolic regression tools
df_2.to_csv("data/burgers_data_with_derivatives.csv", index=False)

In [17]:
df_2

,t,x,u,u_t,u_x,u_xx
0,0.0,-8.0000,2.220446e-16,1.179612e-15,-5.197957e-09,1.874079e-07
1,0.0,-7.9375,-4.310080e-11,1.717322e-09,3.818724e-09,1.011259e-07
2,0.0,-7.8750,4.773408e-10,-1.912823e-08,7.442777e-09,8.615683e-08
3,0.0,-7.8125,8.872463e-10,-3.554737e-08,1.458833e-08,-1.031682e-07
4,0.0,-7.7500,2.300882e-09,-9.211231e-08,-5.453247e-09,-3.188157e-07
...,...,...,...,...,...,...
25851,10.0,7.6875,1.424345e-04,2.074320e-04,1.147614e-04,5.809782e-05
25852,10.0,7.7500,1.497234e-04,2.162733e-04,1.183847e-04,5.769673e-05
25853,10.0,7.8125,1.572326e-04,2.250676e-04,1.219735e-04,5.787001e-05
25854,10.0,7.8750,1.649701e-04,2.340832e-04,1.256184e-04,5.826516e-05


In [18]:
from itertools import combinations_with_replacement

In [19]:
# Define features and interactions
features = ['u', 'u_x', 'u_xx']
target = df_2['u_t']  # Target variable
feature_matrix = pd.DataFrame()

In [20]:
# Add raw features
for feature in features:
    feature_matrix[feature] = df_2[feature]

In [21]:
# Add interaction terms
for degree in range(2, 4):  # Include up to degree-3 interactions
    for combo in combinations_with_replacement(features, degree):
        name = '*'.join(combo)
        term = df_2[combo[0]]
        for var in combo[1:]:
            term *= df_2[var]
        feature_matrix[name] = term

In [22]:
# Save for PyRegressor
feature_matrix['u_t'] = target
feature_matrix.to_csv("data/prepared_data.csv", index=False)

In [23]:
# Load the prepared dataset
data_2 = pd.read_csv("data/prepared_data.csv")
Xf = data_2.drop(columns=['u_t']).values  # Features
u_t_pred = data_2['u_t'].values  # Target

In [36]:
# Train the symbolic regression model

model = PySRRegressor(
    niterations=1000,  # Number of iterations (adjust as necessary)
    
    # List of binary operators (two inputs)
    binary_operators=[
        "+",    # Addition
        "-"    # Subtraction 
    ],
    
    # List of unary operators (one input)
    unary_operators=[
        "sin",      # Sine function
        "cos"      # Cosine function
    ],
    
    # Define any custom operators (optional)
    extra_sympy_mappings={"inv": lambda x: 1/x},  # Example of a custom operator
    variable_names=[f"x{i}" for i in range(X.shape[1])],
    model_selection='accuracy',                    # Enable model selection mode
    population_size=1000,               # Population size for evolutionary search
    maxsize=20                          # Maximum complexity of equations
)
    





c:\Users\chidi\anaconda3\envs\boot_DA\lib\site-packages\pysr\sr.py:922: FutureWarning: `variable_names` is a data-dependent parameter and should be passed when fit is called. Ignoring parameter; please pass `variable_names` during the call to fit instead.
  warnings.warn(


In [ ]:
model.fit(Xf, yf)

c:\Users\chidi\anaconda3\envs\boot_DA\lib\site-packages\pysr\sr.py:2582: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
c:\Users\chidi\anaconda3\envs\boot_DA\lib\site-packages\pysr\sr.py:2029: UserWarning: Note: you are running with more than 10,000 datapoints. You should consider turning on batching (https://astroautomata.com/PySR/options/#batching). You should also reconsider if you need that many datapoints. Unless you have a large amount of noise (in which case you should smooth your dataset first), generally < 10,000 datapoints is enough to find a functional form with symbolic regression. More datapoints will lower the search speed.
  warnings.warn(
c:\Users\chidi\anaconda3\envs\boot_DA\lib\site-packages\pysr\sr.py:2059: UserWarning: Note: you are running with 10 features or more. Genetic algorithms like used in PySR scale poorly with large numbers of features. You should run PySR for more `niterations` to ensure 

Compiling Julia backend...


[ Info: Note: you are running with more than 10,000 datapoints. You should consider turning on batching (`options.batching`), and also if you need that many datapoints. Unless you have a large amount of noise (in which case you should smooth your dataset first), generally < 10,000 datapoints is enough to find a functional form.
[ Info: Started!



Expressions evaluated per second: 1.360e+05
Head worker occupation: 15.4%
Progress: 10 / 15000 total iterations (0.067%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
1           1.664e-02  1.594e+01  y = x₆
2           9.355e-03  5.757e-01  y = sin(-1.28e-05)
3           1.387e-03  1.908e+00  y = x₄ - x₁
4           1.370e-03  1.301e-02  y = sin(x₄) - x₁
5           1.359e-03  7.453e-03  y = sin(sin(x₄)) - x₁
6           1.353e-03  4.333e-03  y = sin(sin(sin(x₄))) - x₁
7           1.350e-03  2.377e-03  y = sin(sin(sin(sin(x₄)))) - x₁
8           1.345e-03  3.821e-03  y = sin(sin(sin(x₄ - -0.0028987))) - x₁
9           1.342e-03  2.284e-03  y = sin(sin(sin(sin(x₄) + 0.0028733))) - x₁
10          1.339e-03  2.517e-03  y = sin(sin(x₁) - (x₁ + (x₁ - sin(x₄))))
11          1.245e-03  7.243e-02  y = sin(sin(sin(sin(sin(sin(x₇ - x₆) - x₁)))))
12          1.195e-03  4.113e-02  y = si

In [2]:
# Load the prepared dataset
data_2 = pd.read_csv("data/prepared_data.csv")
Xf = data_2.drop(columns=['u_t']).values  # Features
u_t_pred = data_2['u_t'].values  # Target

NameError: name 'pd' is not defined

In [4]:
# Load the prepared dataset
data_3 = pd.read_csv("data/burgers_data_with_derivatives.csv")
Xf = data_3.drop(columns=['u_t'])  # Features
u_t_pred = data_3['u_t']  # Target

In [5]:
# Train the symbolic regression model
# Generate variable names based on the feature columns
variable_names = data_3.drop(columns=['u_t']).columns.tolist()

complexity_options = {
    "complexity_of_operators": {
        "+": 5,  # Discourage repeated usage of `+`
        "*": 1,  # Allow multiplication since it's integral to the structure
        "-": 1,  # Allow subtraction
        "/": 10,  # Strongly discourage division unless necessary
        "sin": 100,  # Exclude unnecessary functions
        "cos": 100
    },
    "complexity_of_constants": 1,  # Ensure constants have minimal impact on complexity
    "complexity_of_variables": 10  # Discourage variable repetition
}


model_4 = PySRRegressor(
    niterations=10,  # Number of iterations (adjust as necessary)
    
    # List of binary operators (two inputs)
    binary_operators=[
        "+",    # Addition
       
        '*'     # Multiplication
    ],

    
    
    # Define any custom operators (optional)
    # extra_sympy_mappings={"inv": lambda x: 1/x},  # Example of a custom operator
    variable_names=variable_names,
    model_selection='accuracy',                    # Enable model selection mode           
    select_k_features=3,
    complexity_of_operators=complexity_options["complexity_of_operators"],
    complexity_of_constants=complexity_options["complexity_of_constants"],
    complexity_of_variables=complexity_options["complexity_of_variables"],
    population_size=50,               # Population size for evolutionary search
    batching=True,
    
    batch_size=5000,                    # Batching for large datapoint
    maxsize=10                          # Maximum complexity of equations
)
    





c:\Users\chidi\anaconda3\envs\boot_DA\lib\site-packages\pysr\sr.py:922: FutureWarning: `variable_names` is a data-dependent parameter and should be passed when fit is called. Ignoring parameter; please pass `variable_names` during the call to fit instead.
  warnings.warn(


In [20]:
model_2.fit(Xf, u_t_pred)

c:\Users\chidi\anaconda3\envs\boot_DA\lib\site-packages\pysr\sr.py:2582: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
c:\Users\chidi\anaconda3\envs\boot_DA\lib\site-packages\pysr\sr.py:2029: UserWarning: Note: you are running with more than 10,000 datapoints. You should consider turning on batching (https://astroautomata.com/PySR/options/#batching). You should also reconsider if you need that many datapoints. Unless you have a large amount of noise (in which case you should smooth your dataset first), generally < 10,000 datapoints is enough to find a functional form with symbolic regression. More datapoints will lower the search speed.
  warnings.warn(
[ Info: Note: you are running with more than 10,000 datapoints. You should consider turning on batching (`options.batching`), and also if you need that many datapoints. Unless you have a large amount of noise (in which case you should smooth your dataset first), genera


Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 1500 total iterations (0.000%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 1500 total iterations (0.000%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 1500 total iterations (0.000%)
Hall of Fame:
-------------

PySRRegressor.equations_ = [
	   pick     score                                  equation      loss  \
	0        0.000000                              1.3205863e-8  0.009355   
	1        0.796941                          -0.8168716 * u_x  0.001900   
	2        0.212913                    (-1.3231493 + u) * u_x  0.001241   
	3  >>>>  0.014261  (u_x + -0.0048151943) * (-1.3239754 + u)  0.001206   
	
	   complexity  
	0           1  
	1           3  
	2           5  
	3           7  
]

In [22]:
model_2.fit(Xf, u_t_pred)

c:\Users\chidi\anaconda3\envs\boot_DA\lib\site-packages\pysr\sr.py:2582: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


Using features ['u' 'u_x' 'u_xx']


[ Info: Started!



Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 1500 total iterations (0.000%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 1500 total iterations (0.000%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 1500 total iterations (0.000%)
Hall of Fame:
-------------

PySRRegressor.equations_ = [
	   pick     score                                           equation  \
	0        0.000000                              -4.116597641710977e-9   
	1        0.707027                        -4.116597641710977e-9 - u_x   
	2        0.089913                          -0.8169180222903233 * u_x   
	3        0.212913                    (-1.3230988555110357 + u) * u_x   
	4  >>>>  0.013883  (u - 1.3263918982470777) * (u_x - 0.0055593724...   
	
	       loss  complexity  
	0  0.009355           1  
	1  0.002275           3  
	2  0.001900           5  
	3  0.001241           7  
	4  0.001207           9  
]

In [25]:
model_2.fit(Xf, u_t_pred)

c:\Users\chidi\anaconda3\envs\boot_DA\lib\site-packages\pysr\sr.py:2582: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


Using features ['u' 'u_x' 'u_xx']


[ Info: Started!



Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 150 total iterations (0.000%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 150 total iterations (0.000%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 150 total iterations (0.000%)
Hall of Fame:
----------------

PySRRegressor.equations_ = [
	   pick     score                                           equation  \
	0        0.000000    -0.00043206835777884347 + 0.0004336714354407268   
	1        0.796941                          -0.8169675835712258 * u_x   
	2        0.212911                    (u + -1.3238582332607747) * u_x   
	3        0.005965  ((-1.3219584289773965 + u) * u_x) + 0.00370930...   
	4        0.006891              ((u * u) + -1.1232415743383184) * u_x   
	5        0.027108  u_x * (-1.3238582332607747 + ((u_xx * 0.076714...   
	6  >>>>  0.009580  (0.004082527073600294 + (0.10205703391943304 *...   
	
	       loss  complexity  
	0  0.009355           3  
	1  0.001900           5  
	2  0.001241           7  
	3  0.001227           9  
	4  0.001210          11  
	5  0.001146          13  
	6  0.001124          15  
]

In [29]:
model_2.fit(Xf, u_t_pred)

c:\Users\chidi\anaconda3\envs\boot_DA\lib\site-packages\pysr\sr.py:2582: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


Using features ['u' 'u_x' 'u_xx']


[ Info: Started!



Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 150 total iterations (0.000%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 150 total iterations (0.000%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 150 total iterations (0.000%)
Hall of Fame:
----------------

PySRRegressor.equations_ = [
	   pick     score                                           equation  \
	0        0.000000                                                  u   
	1        1.678395                                   -0.8165916 * u_x   
	2        0.212913                             (u + -1.3231047) * u_x   
	3        0.012892                       u_x * (-1.1264398 + (u * u))   
	4        0.111993    (-1.2940938 + u) * (u_x + (-0.10937756 * u_xx))   
	5        0.004099  ((-1.2972188 + u) * (u_x + (-0.10883774 * u_xx...   
	6        0.050442  (u_x * (-1.3797823 + u)) + (-0.4118863 * (u * ...   
	7  >>>>  0.001165  (u_x * (-0.9722346 + u)) + (-0.40728274 * ((u ...   
	
	       loss  complexity  
	0  0.054533           1  
	1  0.001900           3  
	2  0.001241           5  
	3  0.001210           7  
	4  0.000967           9  
	5  0.000959          11  
	6  0.000867          13  
	7  0.000865          15  
]

In [32]:
model_2.fit(Xf, u_t_pred)

c:\Users\chidi\anaconda3\envs\boot_DA\lib\site-packages\pysr\sr.py:2582: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


Using features ['u' 'u_x' 'u_xx']


[ Info: Started!



Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 150 total iterations (0.000%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 150 total iterations (0.000%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 150 total iterations (0.000%)
Hall of Fame:
----------------

PySRRegressor.equations_ = [
	   pick     score                                           equation  \
	0        0.000000                                  u_x * -0.81700236   
	1        0.212913                             u_x * (-1.3229705 + u)   
	2        0.012892                       ((u * u) + -1.1266235) * u_x   
	3        0.076028    (u_x * (u + -1.3354518)) + (0.057956796 * u_xx)   
	4        0.024713  (u_x * ((u * 1.1503634) + -1.3354518)) + (0.05...   
	5        0.001062  (u_x * ((u * (0.7685667 + 0.36573505)) + -1.33...   
	6  >>>>  0.058375  ((((u * u) * -1.6370099) * 0.11099498) + (u_x ...   
	
	       loss  complexity  
	0  0.001900           3  
	1  0.001241           5  
	2  0.001210           7  
	3  0.001039           9  
	4  0.000989          11  
	5  0.000987          13  
	6  0.000878          15  
]

In [11]:
model_3.fit(Xf, u_t_pred)

c:\Users\chidi\anaconda3\envs\boot_DA\lib\site-packages\pysr\sr.py:2582: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


Using features ['u' 'u_x' 'u_xx']

Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 150 total iterations (0.000%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.


[ Info: Started!



Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 150 total iterations (0.000%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 150 total iterations (0.000%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 150 total iterations (0.000%)
Hall of Fame:
----------------

PySRRegressor.equations_ = [
	   pick     score                                           equation  \
	0        0.000000                                        4.170899e-5   
	1        0.796940                                   -0.8174196 * u_x   
	2        0.212910                             (-1.3241293 + u) * u_x   
	3        0.013400             (u_x + -0.005311489) * (u + -1.336175)   
	4        0.026207     ((u + -1.3454869) + (u_xx * 0.08987158)) * u_x   
	5  >>>>  0.004271  (u_x * ((-1.1897424 + (0.06618561 * u_xx)) + u...   
	
	       loss  complexity  
	0  0.009355           1  
	1  0.001900           3  
	2  0.001241           5  
	3  0.001209           7  
	4  0.001147           9  
	5  0.001137          11  
]

In [9]:
model_2.fit(Xf, u_t_pred)

c:\Users\chidi\anaconda3\envs\boot_DA\lib\site-packages\pysr\sr.py:2582: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


Using features ['u' 'u_x' 'u_xx']
Compiling Julia backend...


[ Info: Started!



Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 150 total iterations (0.000%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 150 total iterations (0.000%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 150 total iterations (0.000%)
Hall of Fame:
----------------

PySRRegressor.equations_ = [
	   pick     score                                           equation  \
	0        0.000000                                                u_x   
	1        1.507883                                   u_x * -0.8166599   
	2        0.212912                             u_x * (u + -1.3236089)   
	3        0.014259            (u_x + -0.004876897) * (-1.3236089 + u)   
	4        0.086747     ((-1.2789557 + u) * u_x) + (0.06074267 * u_xx)   
	5  >>>>  0.017479  ((u_x * (u + -1.1701376)) + (u_xx * 0.04623481...   
	
	       loss  complexity  
	0  0.038776           1  
	1  0.001900           3  
	2  0.001241           5  
	3  0.001206           7  
	4  0.001014           9  
	5  0.000979          11  
]

In [43]:
# Train the symbolic regression model
# Generate variable names based on the feature columns
variable_names = data_3.drop(columns=['u_t']).columns.tolist()

complexity_options = {
    "complexity_of_operators": {
        "+": 2,  # Discourage repeated usage of `+`
        "*": 1,  # Allow multiplication since it's integral to the structure
        "-": 1,  # Allow subtraction
        "/": 10,  # Strongly discourage division unless necessary
        "sin": 100,  # Exclude unnecessary functions
        "cos": 100,
        "()": 6
    },
    "complexity_of_constants": 6,  # Ensure constants have minimal impact on complexity
    "complexity_of_variables": 6  # Discourage variable repetition
}


model_5 = PySRRegressor(
    niterations=2,  # Number of iterations (adjust as necessary)
    
    # List of binary operators (two inputs)
    binary_operators=[
        "+",    # Addition
       
        '*'     # Multiplication
    ],

    
    
    # Define any custom operators (optional)
    # extra_sympy_mappings={"inv": lambda x: 1/x},  # Example of a custom operator
    variable_names=variable_names,
    model_selection='accuracy',                    # Enable model selection mode           
    select_k_features=3,
    complexity_of_operators=complexity_options["complexity_of_operators"],
    complexity_of_constants=complexity_options["complexity_of_constants"],
    complexity_of_variables=complexity_options["complexity_of_variables"],
    population_size=15,               # Population size for evolutionary search
    batching=True,
    
    batch_size=5000,                    # Batching for large datapoint
    maxsize=13                          # Maximum complexity of equations
)
    


c:\Users\chidi\anaconda3\envs\boot_DA\lib\site-packages\pysr\sr.py:922: FutureWarning: `variable_names` is a data-dependent parameter and should be passed when fit is called. Ignoring parameter; please pass `variable_names` during the call to fit instead.
  warnings.warn(


In [ ]:
model_5.fit(Xf, u_t_pred)

c:\Users\chidi\anaconda3\envs\boot_DA\lib\site-packages\pysr\sr.py:2582: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


Using features ['u' 'u_x' 'u_xx']


In [19]:
model_5.fit(Xf, u_t_pred)

c:\Users\chidi\anaconda3\envs\boot_DA\lib\site-packages\pysr\sr.py:2582: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


Using features ['u' 'u_x' 'u_xx']


[ Info: Started!



Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 150 total iterations (0.000%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 4.970e+00
Head worker occupation: 15.3%
Progress: 12 / 150 total iterations (8.000%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
11          2.203e-02  1.449e+00  y = (u * u_xx) * (u * u)
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 4.390e+00
Head worker occupation: 0.0%
Progress: 13 

PySRRegressor.equations_ = [
	   pick  score                  equation      loss  complexity
	0  >>>>    0.0  u_x * (u_x * (u * u_xx))  0.010137          11
]

In [10]:
model_5.fit(Xf, u_t_pred)

c:\Users\chidi\anaconda3\envs\boot_DA\lib\site-packages\pysr\sr.py:2582: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


Using features ['u' 'u_x' 'u_xx']


[ Info: Started!



Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 150 total iterations (0.000%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 150 total iterations (0.000%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 150 total iterations (0.000%)
Hall of Fame:
----------------

PySRRegressor.equations_ = [
	   pick     score                                        equation      loss  \
	0        0.000000                                             u_x  0.038776   
	1        0.423090                                       u_x * u_x  0.016637   
	2        2.169586                                -0.8170132 * u_x  0.001900   
	3        0.212912                          (u + -1.3237236) * u_x  0.001241   
	4        0.012893                    (-1.1265869 + (u * u)) * u_x  0.001210   
	5        0.002658        (-0.0046542743 + u_x) * (-1.3263141 + u)  0.001207   
	6  >>>>  0.087044  (0.060118295 * u_xx) + ((u + -1.286367) * u_x)  0.001014   
	
	   complexity  
	0           1  
	1           3  
	2           4  
	3           6  
	4           8  
	5           9  
	6          11  
]

In [12]:
model_5.fit(Xf, u_t_pred)

c:\Users\chidi\anaconda3\envs\boot_DA\lib\site-packages\pysr\sr.py:2582: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


Using features ['u' 'u_x' 'u_xx']


[ Info: Started!



Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 150 total iterations (0.000%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 150 total iterations (0.000%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 150 total iterations (0.000%)
Hall of Fame:
----------------

PySRRegressor.equations_ = [
	   pick     score                                           equation  \
	0        0.000000             (-1.0246991 * 0.7947002) * 0.011110781   
	1  >>>>  0.002876  -0.00361175 * ((1.2979156 + -0.40752104) * 0.2...   
	
	       loss  complexity  
	0  0.009437           8  
	1  0.009356          11  
]

In [17]:
model_5.fit(Xf, u_t_pred)

c:\Users\chidi\anaconda3\envs\boot_DA\lib\site-packages\pysr\sr.py:2582: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


Using features ['u' 'u_x' 'u_xx']


[ Info: Started!



Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 150 total iterations (0.000%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 1.280e+03
Head worker occupation: 9.2%
Progress: 15 / 150 total iterations (10.000%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 9.680e+02
Head worker occupation: 0.0%
Progress: 24 / 150 total iterations (16.000%)
Hall of Fame:
------------

ValueError: attempt to get argmin of an empty sequence

ValueError: attempt to get argmin of an empty sequence

ValueError: attempt to get argmin of an empty sequence